In [257]:
import sys
sys.path.append("../backend")

In [258]:
not_matching = ['faf4435d-236c-4f96-b2e9-67a8f1c54a97',
              '0edd772e-5cd5-468d-b43f-8c24619bb074',
              'e74bffa4-eef0-4230-9607-7c11994e97cb',
              'a05ed403-9305-4a50-b0d7-07995849d2d0',
              '7fce4656-92ee-4c6f-85d4-6d13dabdbff6',
              '80e19fa7-630a-4acf-8d4e-fd0d5cfd8b74',
              '2b5d2ceb-5804-449a-b9b1-056a41839ad9',
              'dbc90034-6cbe-492f-981f-ec046c250fad',
              'aed5e05d-7315-4051-ab0b-6b73d5319420',
              '13e3d2e8-80c6-4287-9d06-5adb7e115b75',
              '28aeb085-d146-4499-b9ab-2f78e84255f3',
              '5b1d10b3-81a4-45e0-8f10-78ade0a1c5a6',
              '6a30bdea-935b-4229-acb4-004033f91f6a',
              '0b7593c8-a51d-4181-9925-1c4923887da0',
              'f8fc1e8e-fe69-44ec-9be3-5535c1fdbd54',
              'f0943b3c-02af-4ec2-ae7b-b3b5e0012db3',
              '6422a85e-362a-4e39-98b1-2b48897a5dbf',
              'eadb1b89-9c1e-4c5c-82cf-c29a4c836701',
              'e8a50210-8176-4fcc-9378-23cccb5da013',
              'f7bbb89e-5215-4408-967d-85f7ea88a0bd',
              '101743d7-6d0f-4802-8c9b-3ac58e1894c0',
              'd2309370-7971-4a79-9b59-0f6b5bc7e9c3',
              '9dedab11-341d-4e57-90af-c0f0911ebfcf',
              '85d7e8fa-7bf6-42fe-9211-8f084dc45b53',
              '8094bc64-a712-4307-a9f1-cfdf7c92648f',
              '9f4efd09-4eb9-4b3b-b08c-87c7934807fd',
              'f7e0b328-afce-49a2-ad52-4d587c125917',
              'd9191db3-13dd-4d60-9bd9-00bc51c7df99',
              '3a2f9205-cca8-42cb-b744-39e2c4a96155',
              'd64ec2d4-ec3f-42d2-b2d4-e80a4bd95b6c',
              '500fb50e-3d6a-4374-89ce-3430fad2e2ad',
              '552922e3-9a90-46af-a25f-e5107db4a263',
              '4c27f5d3-1830-4a9d-a0f1-63f2fad611ae',
              '64763cbe-f956-4377-9691-9b19b54d606d',
              'b5727112-2caf-4444-b85f-70e7e324d1f7',
              '2563c8ee-74b8-4de4-80f7-5398d35f5edf']

In [259]:
DATASET = "train"
DATASET_FILE = f"../data/generated/{DATASET}.json"
OUTPUT_FILE = f"../data/generated/{DATASET}.tsv"

In [260]:
print(DATASET_FILE)

../data/generated/train.json


In [261]:
import json
import csv
import nltk
import tiktoken
from nltk.tokenize import word_tokenize
from transformers import BertTokenizer, LongformerTokenizer, T5Tokenizer, RobertaTokenizer

# Initialize the tokenizer
# tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
# tokenizer = LongformerTokenizer.from_pretrained('allenai/longformer-base-4096')
tokenizer = T5Tokenizer.from_pretrained('t5-base')
# tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

# Ensure you have the necessary NLTK data files
nltk.download('punkt')
token_encoding = tiktoken.get_encoding("cl100k_base")

def tokenize_long_text(text, max_length=512, stride=50):
    tokens = []
    for i in range(0, len(text), max_length - stride):
        chunk = text[i:i + max_length]
        tokenized_chunk = tokenizer.encode(chunk, add_special_tokens=False)
        tokens.extend(tokenized_chunk)
    return tokens

def create_tsv_from_json(json_file, tsv_file):
    count_not_found = 0
    
    with open(json_file, 'r', encoding='utf-8') as f:
        data = json.load(f)['data']

    with open(tsv_file, 'w', encoding='utf-8', newline='') as tsvfile:
        writer = csv.writer(tsvfile, delimiter='\t')
        writer.writerow(['question', 'tokenized_context', 'tag_annotations'])

        for item in data:
            for paragraph in item['paragraphs']:
                context = paragraph['context'].replace('‘', ' ').replace('’', ' ').replace('\n', ' ')
                # tokenized_context = word_tokenize(context)
                # tokenized_context = token_encoding.encode(context)
                # tokenized_context = tokenize_long_text(context)
                tokenized_context = tokenizer.encode(context, add_special_tokens=False)
                for qa in paragraph['qas']:
                    question = qa['question']
                    id = qa['id']
                    answer_text = qa['answers'][0]['text'].replace('‘', ' ').replace('’', ' ').replace('\n', ' ')
                    if id in not_matching:
                        # set first character to lowercase
                        answer_text = answer_text[0].lower() + answer_text[1:]
                    # tokenized_answer = word_tokenize(answer_text)
                    # tokenized_answer = token_encoding.encode(answer_text)
                    # tokenized_answer = tokenize_long_text(answer_text)
                    tokenized_answer = tokenizer.encode(answer_text, add_special_tokens=False)
        
                    # Create tag annotations
                    found = False
                    annotations = ['O'] * len(tokenized_context)
                    for i in range(len(tokenized_context)):
                        if tokenized_context[i:i + len(tokenized_answer)] == tokenized_answer:
                            found = True
                            annotations[i] = 'B'
                            for j in range(1, len(tokenized_answer)):
                                annotations[i + j] = 'I'
                            break
                    
                    if found:
                        #writer.writerow([question, ' '.join(tokenized_context), ' '.join(annotations)])
                        writer.writerow([question, tokenized_context, annotations])
                    else:
                        print(f"Answer not found in context for id: {id}")
                        count_not_found += 1

    print(f"Total answers not found in context: {count_not_found}")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\onan\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [262]:
create_tsv_from_json(DATASET_FILE, OUTPUT_FILE)

Token indices sequence length is longer than the specified maximum sequence length for this model (901 > 512). Running this sequence through the model will result in indexing errors


Total answers not found in context: 0
